# Data Parsing

The purpose of this notebook is to parse through every NBA games HTML file. Before parsing, each HTML file contains a significant amount of unecessary information. We only care about parsing the line score, the teams playing, and the basic/advanced stats. Using custom functions, we will modify each HTML file, and only extract relevant information required for Machine Learning.

### Import Frameworks + Libraries

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import os

from helper_functions import save_df

### Pointer Logic

- Create pointer variables to save .html files to a particular directory

In [ ]:
SCORE_DIR = "data/scores"
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

### Parse HTML Function

- Takes in a particular HTML file for a specific game, and removes excess table data that is not required in the df (s.decompose())

In [ ]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)

    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]

    return soup
    

### Parse Line Score

- Takes in a BeautifulSoup object to parse the total points/teams playing

In [ ]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols

    line_score = line_score[["team", "total"]]
    return line_score

### Parse 4 Factors

- Takes in a BeautifulSoup object to parse the 4 factors table for each team

In [ ]:
def read_four_factors(soup):
    four_factors = pd.read_html(str(soup), attrs={"id": "four_factors"})[0]
    exlude = ["Pace", "ORtg", "TOV%"]
    four_factors.drop(exlude, axis=1, inplace=True)
    
    cols = list(four_factors.columns)
    cols[0] = "team"
    four_factors.columns = cols

    return four_factors
    

### Read Stats

- Reads in the boxscore HTML table and converts it to a dataframe using Pandas

In [ ]:
def read_stats(soup, team, stat) -> pd.DataFrame:
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")

    return df

### Read Season Info

- Returns the particular season info of the game we are currently parsing

In [ ]:
def read_season_info(soup) -> pd.DataFrame:
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

### Begin Processing

- This is the crux of the parse_data file
- We will iterate through every single HTML file (ie. every single game) and parse the date, season, line score, 4 factors, basic stats, and advanced stats for each team

In [ ]:
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    four_factors = read_four_factors(soup)
    four_factors.set_index('team', inplace=True)
    
    teams = list(line_score["team"])
    summaries = []
    team_mapping = {}

    for i, team in enumerate(teams):
        team_mapping[f"team{i}"] = team
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        # only take the final row in the DataFrame and concatenate basic and advanced stats 
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])

        team_factors = four_factors.loc[team]
        totals = pd.concat([totals, team_factors])

        totals.index = totals.index.str.lower() + f"_team{i}"
            
        totals[f"team{i}"] = team
        summaries.append(totals)

    game = pd.concat(summaries, axis=0)
    game["winner"] = team_mapping["team0"] if game["pts_team0"] > game["pts_team1"] else team_mapping["team1"]
    game = game.to_frame().T

    game["season"] = read_season_info(soup)
    game["date"] = os.path.basename(box_score)[:8]
    game["date"] = pd.to_datetime(game["date"], format="%Y%m%d")
    games.append(game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

### Drop Unnecessary Columns

Due to some discrepancies in older season data, the BPM stat is unavailable for earlier seasons. As a result, during the scraping, we see that certain columns are "unnamed." To account for this, we drop these columns so we can accordingly create a dataframe.

In [ ]:
for game in games:
    columns = game.columns
    
    if "unnamed: 16_team1" in columns:
        game.drop("unnamed: 16_team1", axis=1, inplace=True)
    if "unnamed: 16_team0" in columns:
        game.drop("unnamed: 16_team0", axis=1, inplace=True)
    if "bpm_team1" in columns:
        game.drop("bpm_team1", axis=1, inplace=True)
    if "bpm_team0" in columns:
        game.drop("bpm_team0", axis=1, inplace=True)

### Create the Dataframe with all Games

In [ ]:
games_df = pd.concat(games, axis=0)

### Export the Dataframe as a CSV

In [ ]:
save_df(games_df, "raw_master_df.csv")